### Precipitation

#### Monthly Statistics

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Precipitation\1raw_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year and month as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in tqdm(np.unique(ds['month'].values), desc=f"Year {year_val}", leave=False):
        ds_month = ds.sel(valid_time=(ds['month'] == month_val))
        if ds_month['valid_time'].size == 0:
            continue

        precip_data = ds_month['tp'].values

        # Compute monthly statistics (same logic as daily before)
        mean = np.nanmean(precip_data, axis=0)
        median = np.nanmedian(precip_data, axis=0)
        std = np.nanstd(precip_data, axis=0)
        max_ = np.nanmax(precip_data, axis=0)
        min_ = np.nanmin(precip_data, axis=0)
        total = np.nansum(precip_data, axis=0)

        # Create Dataset for this month's stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_),
            'Total_Precipitation': (['latitude', 'longitude'], total)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        stats_df['Year'] = year_val
        stats_df['Month'] = int(month_val)

        # Set multi-index
        stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"monthly_precipitation_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:22<07:47, 22.28s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly\monthly_precipitation_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly\monthly_precipitation_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [06:44<00:18, 18.80s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly\monthly_precipitation_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [07:21<00:00, 20.05s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly\monthly_precipitation_stats_2024.nc

🎉 All yearly files saved successfully!


Convert lat and lon from 0.25 to 0.75

In [2]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

# Input/output folders
input_folder = r"D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly"
output_folder = r"D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly_regridded"
os.makedirs(output_folder, exist_ok=True)

# Get list of input NetCDF files
file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".nc")])
print(f"Found {len(file_list)} yearly files to regrid...")

# Regrid parameters
factor_lat = 3
factor_lon = 3

# Target grid
lat_target = np.arange(34.5, 66.0 + 0.001, 0.75)   # 43 lats
lon_target = np.arange(-12.0, 36.0 + 0.001, 0.75)  # 65 lons

# Loop through files
for file in tqdm(file_list, desc="Regridding yearly files"):
    file_path = os.path.join(input_folder, file)
    year_val = file.split('_')[-1][:4]  # extract year from filename
    
    # Open dataset
    ds = xr.open_dataset(file_path)
    
    # Step 1: Coarsen (aggregation)
    ds_coarse = xr.Dataset()
    for var in ["Mean", "Median", "Std", "Max", "Min", "Total_Precipitation"]:
        if var in ds:
            ds_coarse[var] = ds[var].coarsen(
                latitude=factor_lat, longitude=factor_lon, boundary="trim"
            ).mean(skipna=True)

    # Handle "number" only if it's in the dataset AND has lat/lon dims
    if "number" in ds and {"latitude", "longitude"}.issubset(ds["number"].dims):
        ds_coarse["number"] = ds["number"].coarsen(
            latitude=factor_lat, longitude=factor_lon, boundary="trim"
        ).sum(skipna=True)

    # Copy non-spatial coords (if they exist)
    for coord in ["Year", "Month"]:
        if coord in ds:
            ds_coarse[coord] = ds[coord]

    # Step 2: Interpolate onto target grid
    ds_final = ds_coarse.interp(
        latitude=lat_target,
        longitude=lon_target,
        method="linear",
        kwargs={"fill_value": "extrapolate"}
    )
    
    # Save output
    out_file = os.path.join(output_folder, f"monthly_precipitation_stats_{year_val}_regrid.nc")
    ds_final.to_netcdf(out_file)
    print(f"✅ Saved {out_file}")

print("\n🎉 All yearly monthly-stat files regridded and saved successfully!")


Found 22 yearly files to regrid...


Regridding yearly files:   5%|▍         | 1/22 [00:00<00:09,  2.30it/s]

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly_regridded\monthly_precipitation_stats_2003_regrid.nc


Regridding yearly files:   9%|▉         | 2/22 [00:00<00:06,  3.06it/s]

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly_regridded\monthly_precipitation_stats_2004_regrid.nc


...

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly_regridded\monthly_precipitation_stats_2022_regrid.nc


Regridding yearly files:  95%|█████████▌| 21/22 [00:05<00:00,  4.09it/s]

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly_regridded\monthly_precipitation_stats_2023_regrid.nc


Regridding yearly files: 100%|██████████| 22/22 [00:05<00:00,  3.94it/s]

✅ Saved D:\IPMA\ERA5\Precipitation\monthly_precipitation_stats_yearly_regridded\monthly_precipitation_stats_2024_regrid.nc

🎉 All yearly monthly-stat files regridded and saved successfully!


#### Daily Statistics

In [3]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Precipitation\1raw_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year, month, day as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month,
        day=ds['valid_time'].dt.day
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in np.unique(ds['month'].values):
        days_in_month = np.unique(ds['day'].values[ds['month'].values == month_val])
        
        # Loop over each day in the month with a progress bar
        for day_val in tqdm(days_in_month, desc=f"Month {month_val}", leave=False):
            ds_day = ds.sel(
                valid_time=(ds['month'] == month_val) & (ds['day'] == day_val)
            )
            if ds_day['valid_time'].size == 0:
                continue
            
            precip_data = ds_day['tp'].values

            # Compute daily statistics
            mean = np.nanmean(precip_data, axis=0)
            median = np.nanmedian(precip_data, axis=0)
            std = np.nanstd(precip_data, axis=0)
            max_ = np.nanmax(precip_data, axis=0)
            min_ = np.nanmin(precip_data, axis=0)
            total = np.nansum(precip_data, axis=0)

            # Create Dataset for this day's stats
            stats = xr.Dataset({
                'Mean': (['latitude', 'longitude'], mean),
                'Median': (['latitude', 'longitude'], median),
                'Std': (['latitude', 'longitude'], std),
                'Max': (['latitude', 'longitude'], max_),
                'Min': (['latitude', 'longitude'], min_),
                'Total_Precipitation': (['latitude', 'longitude'], total)
            }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

            # Convert to DataFrame
            stats_df = stats.to_dataframe().reset_index()

            # Add time labels
            stats_df['Year'] = year_val
            stats_df['Month'] = month_val
            stats_df['Day'] = int(day_val)

            # Set multi-index
            stats_df = stats_df.set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
            df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"daily_precipitation_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [01:44<36:39, 104.76s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly\daily_precipitation_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly\daily_precipitation_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [39:53<01:52, 112.06s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly\daily_precipitation_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [45:31<00:00, 124.14s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly\daily_precipitation_stats_2024.nc

🎉 All yearly files saved successfully!


Convert lat and lon from 0.25 to 0.75

In [4]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

# Input/output folders
input_folder = r"D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly"
output_folder = r"D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly_regridded"
os.makedirs(output_folder, exist_ok=True)

# Get list of input NetCDF files
file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".nc")])
print(f"Found {len(file_list)} yearly files to regrid...")

# Regrid parameters
factor_lat = 3
factor_lon = 3

# Target grid
lat_target = np.arange(34.5, 66.0 + 0.001, 0.75)   # 43 lats
lon_target = np.arange(-12.0, 36.0 + 0.001, 0.75)  # 65 lons

# Loop through files
for file in tqdm(file_list, desc="Regridding yearly files"):
    file_path = os.path.join(input_folder, file)
    year_val = file.split('_')[-1][:4]  # extract year from filename
    
    # Open dataset
    ds = xr.open_dataset(file_path)
    
    # Step 1: Coarsen (aggregation)
    ds_coarse = xr.Dataset()
    for var in ["Mean", "Median", "Std", "Max", "Min", "Total_Precipitation"]:
        if var in ds:
            ds_coarse[var] = ds[var].coarsen(
                latitude=factor_lat, longitude=factor_lon, boundary="trim"
            ).mean(skipna=True)

    # Handle "number" only if it's in the dataset AND has lat/lon dims
    if "number" in ds and {"latitude", "longitude"}.issubset(ds["number"].dims):
        ds_coarse["number"] = ds["number"].coarsen(
            latitude=factor_lat, longitude=factor_lon, boundary="trim"
        ).sum(skipna=True)


    # Copy non-spatial coords (if they align)
    for coord in ["Year", "Month", "Day"]:
        if coord in ds:
            ds_coarse[coord] = ds[coord]
    
    # Step 2: Interpolate onto target grid
    ds_final = ds_coarse.interp(
        latitude=lat_target,
        longitude=lon_target,
        method="linear",
        kwargs={"fill_value": "extrapolate"}
    )
    
    # Save output
    out_file = os.path.join(output_folder, f"daily_precipitation_stats_{year_val}_regrid.nc")
    ds_final.to_netcdf(out_file)
    print(f"✅ Saved {out_file}")

print("\n🎉 All yearly files regridded and saved successfully!")


Found 22 yearly files to regrid...


Regridding yearly files:   5%|▍         | 1/22 [00:06<02:07,  6.07s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly_regridded\daily_precipitation_stats_2003_regrid.nc


Regridding yearly files:   9%|▉         | 2/22 [00:11<01:52,  5.62s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly_regridded\daily_precipitation_stats_2004_regrid.nc


...

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly_regridded\daily_precipitation_stats_2022_regrid.nc


Regridding yearly files:  95%|█████████▌| 21/22 [01:53<00:05,  5.17s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly_regridded\daily_precipitation_stats_2023_regrid.nc


Regridding yearly files: 100%|██████████| 22/22 [01:58<00:00,  5.38s/it]

✅ Saved D:\IPMA\ERA5\Precipitation\daily_precipitation_stats_yearly_regridded\daily_precipitation_stats_2024_regrid.nc

🎉 All yearly files regridded and saved successfully!


### Temperature

#### Monthly Statistics

In [5]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Temperature\1raw_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year and month as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in tqdm(np.unique(ds['month'].values), desc=f"Year {year_val}", leave=False):
        ds_month = ds.sel(valid_time=(ds['month'] == month_val))
        if ds_month['valid_time'].size == 0:
            continue

        temp_data = ds_month['t2m'].values

        # Compute monthly statistics (same logic as daily before)
        mean = np.nanmean(temp_data, axis=0)
        median = np.nanmedian(temp_data, axis=0)
        std = np.nanstd(temp_data, axis=0)
        max_ = np.nanmax(temp_data, axis=0)
        min_ = np.nanmin(temp_data, axis=0)
        total = np.nansum(temp_data, axis=0)

        # Create Dataset for this month's stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        stats_df['Year'] = year_val
        stats_df['Month'] = int(month_val)

        # Set multi-index
        stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"monthly_temperature_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:21<07:35, 21.67s/it]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [08:21<00:23, 23.89s/it]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [09:03<00:00, 24.70s/it]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly\monthly_temperature_stats_2024.nc

🎉 All yearly files saved successfully!


Convert lat and lon from 0.25 to 0.75

In [6]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

# Input/output folders
input_folder = r"D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly"
output_folder = r"D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded"
os.makedirs(output_folder, exist_ok=True)

# Get list of input NetCDF files
file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".nc")])
print(f"Found {len(file_list)} yearly files to regrid...")

# Regrid parameters
factor_lat = 3
factor_lon = 3

# Target grid
lat_target = np.arange(34.5, 66.0 + 0.001, 0.75)   # 43 lats
lon_target = np.arange(-12.0, 36.0 + 0.001, 0.75)  # 65 lons

# Loop through files
for file in tqdm(file_list, desc="Regridding yearly files"):
    file_path = os.path.join(input_folder, file)
    year_val = file.split('_')[-1][:4]  # extract year from filename
    
    # Open dataset
    ds = xr.open_dataset(file_path)
    
    # Step 1: Coarsen (aggregation)
    ds_coarse = xr.Dataset()
    for var in ["Mean", "Median", "Std", "Max", "Min"]:
        if var in ds:
            ds_coarse[var] = ds[var].coarsen(
                latitude=factor_lat, longitude=factor_lon, boundary="trim"
            ).mean(skipna=True)

    # Handle "number" only if it's in the dataset AND has lat/lon dims
    if "number" in ds and {"latitude", "longitude"}.issubset(ds["number"].dims):
        ds_coarse["number"] = ds["number"].coarsen(
            latitude=factor_lat, longitude=factor_lon, boundary="trim"
        ).sum(skipna=True)

    # Copy non-spatial coords (if they exist)
    for coord in ["Year", "Month"]:
        if coord in ds:
            ds_coarse[coord] = ds[coord]

    # Step 2: Interpolate onto target grid
    ds_final = ds_coarse.interp(
        latitude=lat_target,
        longitude=lon_target,
        method="linear",
        kwargs={"fill_value": "extrapolate"}
    )
    
    # Save output
    out_file = os.path.join(output_folder, f"monthly_temperature_stats_{year_val}_regrid.nc")
    ds_final.to_netcdf(out_file)
    print(f"✅ Saved {out_file}")

print("\n🎉 All yearly monthly-stat files regridded and saved successfully!")


Found 22 yearly files to regrid...


Regridding yearly files:   5%|▍         | 1/22 [00:00<00:09,  2.21it/s]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2003_regrid.nc


Regridding yearly files:   9%|▉         | 2/22 [00:00<00:07,  2.72it/s]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2004_regrid.nc


...

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2018_regrid.nc
✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2019_regrid.nc
✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2020_regrid.nc


Regridding yearly files:  95%|█████████▌| 21/22 [00:03<00:00, 10.35it/s]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2021_regrid.nc
✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2022_regrid.nc
✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2023_regrid.nc


Regridding yearly files: 100%|██████████| 22/22 [00:03<00:00,  5.90it/s]

✅ Saved D:\IPMA\ERA5\Temperature\monthly_temperature_stats_yearly_regridded\monthly_temperature_stats_2024_regrid.nc

🎉 All yearly monthly-stat files regridded and saved successfully!


#### Daily Statistics

In [7]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\Temperature\2conversion_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year, month, day as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month,
        day=ds['valid_time'].dt.day
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in np.unique(ds['month'].values):
        days_in_month = np.unique(ds['day'].values[ds['month'].values == month_val])
        
        # Loop over each day in the month with a progress bar
        for day_val in tqdm(days_in_month, desc=f"Month {month_val}", leave=False):
            ds_day = ds.sel(
                valid_time=(ds['month'] == month_val) & (ds['day'] == day_val)
            )
            if ds_day['valid_time'].size == 0:
                continue
            
            temp_data = ds_day['t2m'].values

            # Compute daily statistics
            mean = np.nanmean(temp_data, axis=0)
            median = np.nanmedian(temp_data, axis=0)
            std = np.nanstd(temp_data, axis=0)
            max_ = np.nanmax(temp_data, axis=0)
            min_ = np.nanmin(temp_data, axis=0)

            # Create Dataset for this day's stats
            stats = xr.Dataset({
                'Mean': (['latitude', 'longitude'], mean),
                'Median': (['latitude', 'longitude'], median),
                'Std': (['latitude', 'longitude'], std),
                'Max': (['latitude', 'longitude'], max_),
                'Min': (['latitude', 'longitude'], min_)
            }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

            # Convert to DataFrame
            stats_df = stats.to_dataframe().reset_index()

            # Add time labels
            stats_df['Year'] = year_val
            stats_df['Month'] = month_val
            stats_df['Day'] = int(day_val)

            # Set multi-index
            stats_df = stats_df.set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
            df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"daily_temperature_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:33<11:45, 33.62s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [11:49<00:34, 34.44s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [12:22<00:00, 33.74s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly\daily_temperature_stats_2024.nc

🎉 All yearly files saved successfully!


Convert lat and lon from 0.25 to 0.75

In [8]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

# Input/output folders
input_folder = r"D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly"
output_folder = r"D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly_regridded"
os.makedirs(output_folder, exist_ok=True)

# Get list of input NetCDF files
file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".nc")])
print(f"Found {len(file_list)} yearly files to regrid...")

# Regrid parameters
factor_lat = 3
factor_lon = 3

# Target grid
lat_target = np.arange(34.5, 66.0 + 0.001, 0.75)   # 43 lats
lon_target = np.arange(-12.0, 36.0 + 0.001, 0.75)  # 65 lons

# Loop through files
for file in tqdm(file_list, desc="Regridding yearly files"):
    file_path = os.path.join(input_folder, file)
    year_val = file.split('_')[-1][:4]  # extract year from filename
    
    # Open dataset
    ds = xr.open_dataset(file_path)
    
    # Step 1: Coarsen (aggregation)
    ds_coarse = xr.Dataset()
    for var in ["Mean", "Median", "Std", "Max", "Min"]:
        if var in ds:
            ds_coarse[var] = ds[var].coarsen(
                latitude=factor_lat, longitude=factor_lon, boundary="trim"
            ).mean(skipna=True)

    # Handle "number" only if it's in the dataset AND has lat/lon dims
    if "number" in ds and {"latitude", "longitude"}.issubset(ds["number"].dims):
        ds_coarse["number"] = ds["number"].coarsen(
            latitude=factor_lat, longitude=factor_lon, boundary="trim"
        ).sum(skipna=True)

    # Copy non-spatial coords (if they align)
    for coord in ["Year", "Month", "Day"]:
        if coord in ds:
            ds_coarse[coord] = ds[coord]
    
    # Step 2: Interpolate onto target grid
    ds_final = ds_coarse.interp(
        latitude=lat_target,
        longitude=lon_target,
        method="linear",
        kwargs={"fill_value": "extrapolate"}
    )
    
    # Save output
    out_file = os.path.join(output_folder, f"daily_temperature_stats_{year_val}_regrid.nc")
    ds_final.to_netcdf(out_file)
    print(f"✅ Saved {out_file}")

print("\n🎉 All yearly temperature files regridded and saved successfully!")


Found 22 yearly files to regrid...


Regridding yearly files:   5%|▍         | 1/22 [00:03<01:23,  3.99s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly_regridded\daily_temperature_stats_2003_regrid.nc


Regridding yearly files:   9%|▉         | 2/22 [00:08<01:21,  4.09s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly_regridded\daily_temperature_stats_2004_regrid.nc


...

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly_regridded\daily_temperature_stats_2022_regrid.nc


Regridding yearly files:  95%|█████████▌| 21/22 [01:29<00:04,  4.50s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly_regridded\daily_temperature_stats_2023_regrid.nc


Regridding yearly files: 100%|██████████| 22/22 [01:34<00:00,  4.28s/it]

✅ Saved D:\IPMA\ERA5\Temperature\daily_temperature_stats_yearly_regridded\daily_temperature_stats_2024_regrid.nc

🎉 All yearly temperature files regridded and saved successfully!


### Wind Speed

#### Wind speed and direction 

In [10]:
#Calculate wind speed and direction based on u&v component

import xarray as xr
import numpy as np
import os
from glob import glob

# Define folders
input_folder = r"D:\IPMA\ERA5\UV_wind\1raw_year_1979_2024"
output_folder = r"D:\IPMA\ERA5\UV_wind\2wind_speed_direction"
os.makedirs(output_folder, exist_ok=True)

# Find all relevant NetCDF files
nc_files = sorted(glob(os.path.join(input_folder, "ERA5_hourly_uv_*.nc")))

for file_path in nc_files:
    print(f"Processing {os.path.basename(file_path)}")

    # Open dataset
    ds = xr.open_dataset(file_path)

    # Calculate wind speed
    wind_speed = np.sqrt(ds['u10']**2 + ds['v10']**2)

    # Calculate wind direction (degrees, meteorological convention)
    wind_dir = (180 + np.degrees(np.arctan2(ds['u10'], ds['v10']))) % 360

    # Add to dataset
    ds = ds.assign(wind_speed=wind_speed, wind_direction=wind_dir)

    # Add metadata
    ds['wind_speed'].attrs['units'] = 'm/s'
    ds['wind_speed'].attrs['description'] = '10m wind speed calculated from u10 and v10'
    ds['wind_direction'].attrs['units'] = 'degrees'
    ds['wind_direction'].attrs['description'] = 'Wind direction (from which wind blows, 0°=North, clockwise)'

    # Create output filename, e.g., ERA5_hourly_wind_1979.nc
    year_str = os.path.basename(file_path).split('_')[-1].split('.')[0]
    out_filename = f"ERA5_hourly_wind_{year_str}.nc"
    out_path = os.path.join(output_folder, out_filename)

    # Save only the wind_speed and wind_direction variables (optional)
    ds[['wind_speed', 'wind_direction']].to_netcdf(out_path)

    ds.close()

print("✅ All files processed and saved.")


Processing ERA5_hourly_uv_1979.nc
Processing ERA5_hourly_uv_1980.nc
Processing ERA5_hourly_uv_1981.nc
Processing ERA5_hourly_uv_1982.nc
Processing ERA5_hourly_uv_1983.nc
Processing ERA5_hourly_uv_1984.nc
Processing ERA5_hourly_uv_1985.nc
Processing ERA5_hourly_uv_1986.nc
Processing ERA5_hourly_uv_1987.nc
Processing ERA5_hourly_uv_1988.nc
Processing ERA5_hourly_uv_1989.nc
Processing ERA5_hourly_uv_1990.nc
Processing ERA5_hourly_uv_1991.nc
Processing ERA5_hourly_uv_1992.nc
Processing ERA5_hourly_uv_1993.nc
Processing ERA5_hourly_uv_1994.nc
Processing ERA5_hourly_uv_1995.nc
Processing ERA5_hourly_uv_1996.nc
Processing ERA5_hourly_uv_1997.nc
Processing ERA5_hourly_uv_1998.nc
Processing ERA5_hourly_uv_1999.nc
Processing ERA5_hourly_uv_2000.nc
Processing ERA5_hourly_uv_2001.nc
Processing ERA5_hourly_uv_2002.nc
Processing ERA5_hourly_uv_2003.nc
Processing ERA5_hourly_uv_2004.nc
Processing ERA5_hourly_uv_2005.nc
Processing ERA5_hourly_uv_2006.nc
Processing ERA5_hourly_uv_2007.nc
Processing ERA

#### Monthly Statistics

In [11]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\UV_wind\2wind_speed_direction"
output_folder = r"D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year and month as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in tqdm(np.unique(ds['month'].values), desc=f"Year {year_val}", leave=False):
        ds_month = ds.sel(valid_time=(ds['month'] == month_val))
        if ds_month['valid_time'].size == 0:
            continue

        wind_data = ds_month['wind_speed'].values

        # Compute daily statistics
        mean = np.nanmean(wind_data, axis=0)
        median = np.nanmedian(wind_data, axis=0)
        std = np.nanstd(wind_data, axis=0)
        max_ = np.nanmax(wind_data, axis=0)
        min_ = np.nanmin(wind_data, axis=0)

        # Create Dataset for this month's stats
        stats = xr.Dataset({
            'Mean': (['latitude', 'longitude'], mean),
            'Median': (['latitude', 'longitude'], median),
            'Std': (['latitude', 'longitude'], std),
            'Max': (['latitude', 'longitude'], max_),
            'Min': (['latitude', 'longitude'], min_)
        }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

        # Convert to DataFrame
        stats_df = stats.to_dataframe().reset_index()

        # Add time labels
        stats_df['Year'] = year_val
        stats_df['Month'] = int(month_val)

        # Set multi-index
        stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"monthly_wind_speed_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:23<08:11, 23.38s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [09:50<00:30, 30.06s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [10:20<00:00, 28.19s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly\monthly_wind_speed_stats_2024.nc

🎉 All yearly files saved successfully!


Convert lat and lon from 0.25 to 0.75

In [12]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

# Input/output folders
input_folder = r"D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly"
output_folder = r"D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded"
os.makedirs(output_folder, exist_ok=True)

# Get list of input NetCDF files
file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".nc")])
print(f"Found {len(file_list)} yearly files to regrid...")

# Regrid parameters
factor_lat = 3
factor_lon = 3

# Target grid
lat_target = np.arange(34.5, 66.0 + 0.001, 0.75)   # 43 lats
lon_target = np.arange(-12.0, 36.0 + 0.001, 0.75)  # 65 lons

# Loop through files
for file in tqdm(file_list, desc="Regridding yearly files"):
    file_path = os.path.join(input_folder, file)
    year_val = file.split('_')[-1][:4]  # extract year from filename
    
    # Open dataset
    ds = xr.open_dataset(file_path)
    
    # Step 1: Coarsen (aggregation)
    ds_coarse = xr.Dataset()
    for var in ["Mean", "Median", "Std", "Max", "Min"]:
        if var in ds:
            ds_coarse[var] = ds[var].coarsen(
                latitude=factor_lat, longitude=factor_lon, boundary="trim"
            ).mean(skipna=True)

    # Handle "number" only if it's in the dataset AND has lat/lon dims
    if "number" in ds and {"latitude", "longitude"}.issubset(ds["number"].dims):
        ds_coarse["number"] = ds["number"].coarsen(
            latitude=factor_lat, longitude=factor_lon, boundary="trim"
        ).sum(skipna=True)

    # Copy non-spatial coords (if they exist)
    for coord in ["Year", "Month"]:
        if coord in ds:
            ds_coarse[coord] = ds[coord]

    # Step 2: Interpolate onto target grid
    ds_final = ds_coarse.interp(
        latitude=lat_target,
        longitude=lon_target,
        method="linear",
        kwargs={"fill_value": "extrapolate"}
    )
    
    # Save output
    out_file = os.path.join(output_folder, f"monthly_wind_speed_stats_{year_val}_regrid.nc")
    ds_final.to_netcdf(out_file)
    print(f"✅ Saved {out_file}")

print("\n🎉 All yearly monthly-stat files regridded and saved successfully!")


Found 22 yearly files to regrid...


Regridding yearly files:   5%|▍         | 1/22 [00:00<00:08,  2.62it/s]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2003_regrid.nc


Regridding yearly files:   9%|▉         | 2/22 [00:00<00:06,  3.28it/s]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2004_regrid.nc


...

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2019_regrid.nc
✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2020_regrid.nc


Regridding yearly files:  91%|█████████ | 20/22 [00:06<00:00,  5.16it/s]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2021_regrid.nc
✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2022_regrid.nc


Regridding yearly files: 100%|██████████| 22/22 [00:06<00:00,  3.24it/s]

✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2023_regrid.nc
✅ Saved D:\IPMA\ERA5\UV_wind\monthly_wind_speed_stats_yearly_regridded\monthly_wind_speed_stats_2024_regrid.nc

🎉 All yearly monthly-stat files regridded and saved successfully!


#### Daily statistics

In [13]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from tqdm import tqdm  # For progress bars

# Define the folder containing NetCDF files
folder_path = r"D:\IPMA\ERA5\UV_wind\2wind_speed_direction"
output_folder = r"D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly"

os.makedirs(output_folder, exist_ok=True)

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Filter only years 2003–2024
file_list = [f for f in file_list if 2003 <= int(f.split('_')[-1][:4]) <= 2024]

print(f"Processing {len(file_list)} files from 2003 to 2024...")

# Loop over files with a progress bar
for file in tqdm(file_list, desc="Files processed"):
    file_path = os.path.join(folder_path, file)
    
    # Extract year from filename
    year_val = int(file.split('_')[-1][:4])
    print(f"\n📂 Processing year {year_val}...")
    
    ds = xr.open_dataset(file_path)  
    ds = ds.chunk({'valid_time': 500})  
    
    # Add year, month, day as coordinates
    ds = ds.assign_coords(
        year=ds['valid_time'].dt.year,
        month=ds['valid_time'].dt.month,
        day=ds['valid_time'].dt.day
    )
    
    # Temporary list for this year
    df_list = []
    
    # Loop over each unique month in this file
    for month_val in np.unique(ds['month'].values):
        days_in_month = np.unique(ds['day'].values[ds['month'].values == month_val])
        
        # Loop over each day in the month with a progress bar
        for day_val in tqdm(days_in_month, desc=f"Month {month_val}", leave=False):
            ds_day = ds.sel(
                valid_time=(ds['month'] == month_val) & (ds['day'] == day_val)
            )
            if ds_day['valid_time'].size == 0:
                continue
            
            wind_data = ds_day['wind_speed'].values

            # Compute daily statistics
            mean = np.nanmean(wind_data, axis=0)
            median = np.nanmedian(wind_data, axis=0)
            std = np.nanstd(wind_data, axis=0)
            max_ = np.nanmax(wind_data, axis=0)
            min_ = np.nanmin(wind_data, axis=0)

            # Create Dataset for this day's stats
            stats = xr.Dataset({
                'Mean': (['latitude', 'longitude'], mean),
                'Median': (['latitude', 'longitude'], median),
                'Std': (['latitude', 'longitude'], std),
                'Max': (['latitude', 'longitude'], max_),
                'Min': (['latitude', 'longitude'], min_)
            }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

            # Convert to DataFrame
            stats_df = stats.to_dataframe().reset_index()

            # Add time labels
            stats_df['Year'] = year_val
            stats_df['Month'] = month_val
            stats_df['Day'] = int(day_val)

            # Set multi-index
            stats_df = stats_df.set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
            df_list.append(stats_df)
    
    # Save this year's results
    if df_list:  # Only save if data exists
        df_final = pd.concat(df_list)
        df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'Day', 'latitude', 'longitude'])
        df_final_xr = df_final_xr.to_xarray()
        
        output_file_path = os.path.join(output_folder, f"daily_wind_speed_stats_{year_val}.nc")
        df_final_xr.to_netcdf(output_file_path)
        print(f"✅ Saved {output_file_path}")

print("\n🎉 All yearly files saved successfully!")


Processing 22 files from 2003 to 2024...


Files processed:   0%|          | 0/22 [00:00<?, ?it/s]


📂 Processing year 2003...


Files processed:   5%|▍         | 1/22 [00:53<18:47, 53.69s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2003.nc

📂 Processing year 2004...


...

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2022.nc

📂 Processing year 2023...


Files processed:  95%|█████████▌| 21/22 [15:55<00:45, 45.52s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2023.nc

📂 Processing year 2024...


Files processed: 100%|██████████| 22/22 [16:34<00:00, 45.22s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly\daily_wind_speed_stats_2024.nc

🎉 All yearly files saved successfully!


Convert lat and lon from 0.25 to 0.75

In [14]:
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

# Input/output folders
input_folder = r"D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly"
output_folder = r"D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly_regridded"
os.makedirs(output_folder, exist_ok=True)

# Get list of input NetCDF files
file_list = sorted([f for f in os.listdir(input_folder) if f.endswith(".nc")])
print(f"Found {len(file_list)} yearly files to regrid...")

# Regrid parameters
factor_lat = 3
factor_lon = 3

# Target grid
lat_target = np.arange(34.5, 66.0 + 0.001, 0.75)   # 43 lats
lon_target = np.arange(-12.0, 36.0 + 0.001, 0.75)  # 65 lons

# Loop through files
for file in tqdm(file_list, desc="Regridding yearly files"):
    file_path = os.path.join(input_folder, file)
    year_val = file.split('_')[-1][:4]  # extract year from filename
    
    # Open dataset
    ds = xr.open_dataset(file_path)
    
    # Step 1: Coarsen (aggregation)
    ds_coarse = xr.Dataset()
    for var in ["Mean", "Median", "Std", "Max", "Min"]:
        if var in ds:
            ds_coarse[var] = ds[var].coarsen(
                latitude=factor_lat, longitude=factor_lon, boundary="trim"
            ).mean(skipna=True)

    # Handle "number" only if it's in the dataset AND has lat/lon dims
    if "number" in ds and {"latitude", "longitude"}.issubset(ds["number"].dims):
        ds_coarse["number"] = ds["number"].coarsen(
            latitude=factor_lat, longitude=factor_lon, boundary="trim"
        ).sum(skipna=True)

    # Copy non-spatial coords (if they align)
    for coord in ["Year", "Month", "Day"]:
        if coord in ds:
            ds_coarse[coord] = ds[coord]
    
    # Step 2: Interpolate onto target grid
    ds_final = ds_coarse.interp(
        latitude=lat_target,
        longitude=lon_target,
        method="linear",
        kwargs={"fill_value": "extrapolate"}
    )
    
    # Save output
    out_file = os.path.join(output_folder, f"daily_wind_speed_stats_{year_val}_regrid.nc")
    ds_final.to_netcdf(out_file)
    print(f"✅ Saved {out_file}")

print("\n🎉 All yearly wind speed files regridded and saved successfully!")


Found 22 yearly files to regrid...


Regridding yearly files:   5%|▍         | 1/22 [00:04<01:43,  4.91s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly_regridded\daily_wind_speed_stats_2003_regrid.nc


Regridding yearly files:   9%|▉         | 2/22 [00:09<01:36,  4.80s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly_regridded\daily_wind_speed_stats_2004_regrid.nc


...

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly_regridded\daily_wind_speed_stats_2022_regrid.nc


Regridding yearly files:  95%|█████████▌| 21/22 [01:32<00:04,  4.03s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly_regridded\daily_wind_speed_stats_2023_regrid.nc


Regridding yearly files: 100%|██████████| 22/22 [01:36<00:00,  4.40s/it]

✅ Saved D:\IPMA\ERA5\UV_wind\daily_wind_speed_stats_yearly_regridded\daily_wind_speed_stats_2024_regrid.nc

🎉 All yearly wind speed files regridded and saved successfully!
